In [3]:
import os
import time
import requests
from tqdm import tqdm

# ===============================
# CONFIGURATION
# ===============================
desktop_path = r"C:\Users\study\OneDrive\Desktop"
save_dir = os.path.join(desktop_path, "Radiosonde_Data")  # Save in subfolder
station_id = "43279"
years = [2022, 2023]
months = [f"{m:02d}" for m in range(1, 13)]
days_in_month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
times = ["00", "12"]

# Make sure folder exists
os.makedirs(save_dir, exist_ok=True)

# ===============================
# GENERATE TASKS
# ===============================
tasks = []
for year in years:
    for month_idx, month in enumerate(months):
        for day in range(1, days_in_month[month_idx] + 1):
            day_str = f"{day:02d}"
            for t in times:
                url = (
                    f"https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST"
                    f"&YEAR={year}&MONTH={month}&FROM={day_str}{t}&TO={day_str}{t}&STNM={station_id}"
                )
                filename = f"{station_id}_{year}_{month}_{day_str}_{t}Z.txt"
                filepath = os.path.join(save_dir, filename)
                tasks.append((url, filepath))

# ===============================
# DOWNLOAD FUNCTION WITH RETRY
# ===============================
def download_file(url, filepath, retries=3, delay=2):
    for attempt in range(retries):
        try:
            response = requests.get(url, timeout=15)
            response.raise_for_status()
            with open(filepath, "wb") as f:
                f.write(response.content)
            return True
        except Exception:
            time.sleep(delay)  # wait before retry
    return False

# ===============================
# SEQUENTIAL DOWNLOAD WITH PROGRESS BAR
# ===============================
success_count = 0
for url, filepath in tqdm(tasks, desc="Downloading", unit="file"):
    if download_file(url, filepath):
        success_count += 1
    time.sleep(0.5)  # small delay to avoid server blocking

print(f"\n✅ Download complete: {success_count}/{len(tasks)} files saved to {save_dir}")


Downloading: 100%|███████████████████████████████████████████████████████████████| 1460/1460 [56:50<00:00,  2.34s/file]


✅ Download complete: 1460/1460 files saved to C:\Users\study\OneDrive\Desktop\Radiosonde_Data
